In [1]:
using Pkg

In [2]:
Pkg.status()

Status `~/.julia/environments/v1.5/Project.toml`
  [0c46a032] DifferentialEquations v6.9.0
  [31c24e10] Distributions v0.21.12
  [38e38edf] GLM v1.3.11
  [7073ff75] IJulia v1.23.1
  [a98d9a8b] Interpolations v0.12.10
  [c7f686f2] MCMCChains v4.5.0
  [6f286f6a] MultivariateStats v0.7.0
  [65888b18] ParameterizedFunctions v4.2.1
  [91a5bcdd] Plots v0.28.4
  [9f7883ad] Tracker v0.2.12
  [fce5fe82] Turing v0.5.1


In [3]:
Pkg.add(Pkg.PackageSpec(;name="DifferentialEquations", version="6.16.0"))

   Updating registry at `~/.julia/registries/General`
tar: Failed to set default locale
tar: Failed to set default locale
  Resolving package versions...
tar: Failed to set default locale
tar: Failed to set default locale
tar: Failed to set default locale
tar: Failed to set default locale
tar: Failed to set default locale
tar: Failed to set default locale
tar: Failed to set default locale
  Installed MCMCChains ────────────── v4.6.0
  Installed ParameterizedFunctions ── v5.9.0
  Installed NonlinearSolve ────────── v0.3.7
  Installed DiffEqJump ────────────── v6.13.0
  Installed ThreadingUtilities ────── v0.2.3
  Installed SymbolicUtils ─────────── v0.7.8
  Installed SLEEFPirates ──────────── v0.6.8
tar: Failed to set default locale
  Installed StochasticDiffEq ──────── v6.32.0
tar: Failed to set default locale
tar: Failed to set default locale
tar: Failed to set default locale
  Installed Adapt ─────────────────── v3.1.1
  Installed RecursiveFactorization ── v0.1.10
  Installed Splitta

In [4]:
Pkg.status()

Status `~/.julia/environments/v1.5/Project.toml`
  [0c46a032] DifferentialEquations v6.16.0
  [31c24e10] Distributions v0.24.12
  [38e38edf] GLM v1.3.11
  [7073ff75] IJulia v1.23.1
  [a98d9a8b] Interpolations v0.13.1
  [c7f686f2] MCMCChains v4.6.0
  [6f286f6a] MultivariateStats v0.8.0
  [65888b18] ParameterizedFunctions v5.9.0
  [91a5bcdd] Plots v1.10.2
  [9f7883ad] Tracker v0.2.15
  [fce5fe82] Turing v0.15.8


In [25]:
using Turing

In [26]:
using DifferentialEquations 

In [27]:
using Plots

In [32]:
u0 = [1.0,1.0]
tspan = (0.0,10.0)
function multiplicative_noise!(du,u,p,t)
  x,y = u
  du[1] = p[5]*x
  du[2] = p[6]*y
end
p = [1.5,1.0,3.0,1.0,0.1,0.1]

function lotka_volterra!(du,u,p,t)
  x,y = u
  α,β,γ,δ = p
  du[1] = dx = α*x - β*x*y
  du[2] = dy = δ*x*y - γ*y
end


prob_sde = SDEProblem(lotka_volterra!,multiplicative_noise!,u0,tspan,p)

ensembleprob = EnsembleProblem(prob_sde)
data = solve(ensembleprob,SOSRI(),saveat=0.1,trajectories=1000)

Turing.setadbackend(:forwarddiff)
@model function fitlv(data, prob)
    σ ~ InverseGamma(2,3)
    α ~ truncated(Normal(1.3,0.5),0.5,2.5)
#     β ~ truncated(Normal(1.2,0.25),0.5,2)
#     γ ~ truncated(Normal(3.2,0.25),2.2,4.0)
#     δ ~ truncated(Normal(1.2,0.25),0.5,2.0)
#     ϕ1 ~ truncated(Normal(0.12,0.3),0.05,0.25)
#     ϕ2 ~ truncated(Normal(0.12,0.3),0.05,0.25)
#     α = 1.5
    β = 1.0
    γ = 3.0
    δ = 1.0
    ϕ1 = 0.1
    ϕ2 = 0.1
    p = [α,β,γ,δ,ϕ1,ϕ2]
    prob = remake(prob, p=p)
    predicted = solve(prob,SOSRI(),saveat=0.1)

    if predicted.retcode != :Success
        Turing.acclogp!(_varinfo, -Inf)
    end
    for j in 1:length(data)
        for i = 1:length(predicted)
            data[j][i] ~ MvNormal(predicted[i],σ)
        end
    end
end;

┌ Warning: you are using the internal variable `_varinfo`
└ @ DynamicPPL /Users/zeynepenkavi/.julia/packages/DynamicPPL/wf0dU/src/compiler.jl:171


In [39]:
model = fitlv(data, prob_sde)
# chain = sample(model, NUTS(0.25), 200, init_theta = [1.5,1.3,1.2,2.7,1.2,0.12,0.12])
chain = sample(model, NUTS(0.25), 100)
# plot(chain)

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/zeynepenkavi/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/zeynepenkavi/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/zeynepenkavi/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/zeynepenkavi/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite

Chains MCMC chain (100×14×1 Array{Float64,3}):

Iterations        = 1:100
Thinning interval = 1
Chains            = 1
Samples per chain = 100
parameters        = α, σ
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, max_hamiltonian_energy_error, n_steps, nom_step_size, numerical_error, step_size, tree_depth

Summary Statistics
  parameters      mean       std   naive_se      mcse       ess      rhat 
      Symbol   Float64   Float64    Float64   Missing   Float64   Float64 

           α    1.5139    0.0252     0.0025   missing   19.2541    1.0526
           σ    1.3914    0.0273     0.0027   missing   10.1979    1.0526

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

           α    1.4682    1.4966    1.5125    1.5287    1.5720
           σ    1.3213    1.3817    1.3929    1.4096    1.4305


In [40]:
@code_warntype model.f(
    Random.GLOBAL_RNG,
    model,
    Turing.VarInfo(model),
    Turing.SampleFromPrior(),
    Turing.DefaultContext(),
    model.args...,
)

Variables
  #self#::Core.Compiler.Const(var"#13#14"(), false)
  _rng::Core.Compiler.Const(Random._GLOBAL_RNG(), false)
  _model::DynamicPPL.Model{var"#13#14",(:data, :prob),(),(),Tuple{EnsembleSolution{Float64,3,Array{RODESolution{Float64,2,Array{Array{Float64,1},1},Nothing,Nothing,Array{Float64,1},NoiseProcess{Float64,2,Float64,Array{Float64,1},Array{Float64,1},Array{Array{Float64,1},1},typeof(DiffEqNoiseProcess.INPLACE_WHITE_NOISE_DIST),typeof(DiffEqNoiseProcess.INPLACE_WHITE_NOISE_BRIDGE),true,ResettableStacks.ResettableStack{Tuple{Float64,Array{Float64,1},Array{Float64,1}},true},ResettableStacks.ResettableStack{Tuple{Float64,Array{Float64,1},Array{Float64,1}},true},RSWM{Float64},Nothing,RandomNumbers.Xorshifts.Xoroshiro128Plus},SDEProblem{Array{Float64,1},Tuple{Float64,Float64},true,Array{Float64,1},Nothing,SDEFunction{true,typeof(lotka_volterra!),typeof(multiplicative_noise!),LinearAlgebra.UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing

In [42]:
function lotka_volterra(du,u,p,t)
  x, y = u
  α, β, γ, δ  = p
  du[1] = (α - β*y)x 
  du[2] = (δ*x - γ)y 
end
p = [1.5, 1.0, 3.0, 1.0]
u0 = [1.0,1.0]
prob1 = ODEProblem(lotka_volterra,u0,(0.0,10.0),p)
sol = solve(prob1,Tsit5())




sol1 = solve(prob1,Tsit5(),saveat=0.1)
odedata = Array(sol1) + 0.8 * randn(size(Array(sol1)))
plot(sol1, alpha = 0.3, legend = false); scatter!(sol1.t, odedata')

# set AD backend
Turing.setadbackend(:forwarddiff)

# set priors
priors = [truncated(Normal(1.5,0.5),0.5,2.5), truncated(Normal(1.2,0.5),0,2), truncated(Normal(3.0,0.5),1,4),truncated(Normal(1.0,0.5),0,2)]

# type stable model

@model function fitlv2(data, prob1, priors, ::Type{T} = Float64) where {T} # data should be a Vector
    σ ~ InverseGamma(2, 3)

    p ~ arraydist(priors) 

    predicted::Vector{Vector{T}} = solve(prob1, Tsit5(); p=p, saveat=0.1).u

    for i = 1:length(predicted)
        data[:,i] ~ MvNormal(predicted[i], σ)
    end
end


# instantiation
model2= fitlv2(odedata, prob1, priors)

DynamicPPL.Model{var"#19#20",(:data, :prob1, :priors, :T),(:T,),(),Tuple{Array{Float64,2},ODEProblem{Array{Float64,1},Tuple{Float64,Float64},true,Array{Float64,1},ODEFunction{true,typeof(lotka_volterra),LinearAlgebra.UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing},Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}},DiffEqBase.StandardODEProblem},Array{Truncated{Normal{Float64},Continuous,Float64},1},Type{Float64}},Tuple{Type{Float64}}}(:fitlv2, var"#19#20"(), NamedTuple{(:data, :prob1, :priors, :T),Tuple{Array{Float64,2},ODEProblem{Array{Float64,1},Tuple{Float64,Float64},true,Array{Float64,1},ODEFunction{true,typeof(lotka_volterra),LinearAlgebra.UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing},Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}},DiffEqBase.StandardODEProblem},Array{Truncated{Normal{Float64},Continuou

In [43]:
chain2 = sample(model2, NUTS(0.25), 100)

┌ Info: Found initial step size
│   ϵ = 0.2
└ @ Turing.Inference /Users/zeynepenkavi/.julia/packages/Turing/O1Pn0/src/inference/hmc.jl:195


Chains MCMC chain (100×17×1 Array{Float64,3}):

Iterations        = 1:100
Thinning interval = 1
Chains            = 1
Samples per chain = 100
parameters        = p[1], p[2], p[3], p[4], σ
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, max_hamiltonian_energy_error, n_steps, nom_step_size, numerical_error, step_size, tree_depth

Summary Statistics
  parameters      mean       std   naive_se      mcse       ess      rhat 
      Symbol   Float64   Float64    Float64   Missing   Float64   Float64 

        p[1]    1.6061    0.0172     0.0017   missing    1.6604    1.6530
        p[2]    1.1641    0.0141     0.0014   missing   10.4073    1.0144
        p[3]    2.7673    0.0329     0.0033   missing    1.6921    1.6297
        p[4]    0.9104    0.0195     0.0020   missing    1.3818    1.9848
           σ    0.7961    0.0259     0.0026   missing   12.7370    0.9924

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%    